# Principle and implementation of feature selection

Data and features determine the upper limit of machine learning, and models and algorithms only approximate this upper limit. Thus, feature engineering plays an important role in machine learning. In practical application, feature engineering is the key to the success of machine learning.

In actual project, we may have a lot of features can be used, some characteristics of rich information, some characteristics of carrying information overlap, some features are irrelevant, if all features all as training without screening, frequently, dimension disaster problem, even will reduce the accuracy of the model. Therefore, we need to carry out feature screening to exclude invalid/redundant features and select useful features as training data of the model.

1. Characteristics are classified according to importance

Related features:
- It is helpful for learning tasks (such as classification problems) and can improve the effect of learning algorithm.

Irrelevant features:
- There is no help for our algorithm, it will not bring any improvement to the effect of the algorithm;

Redundancy features:
- Does not bring new information to our algorithm, or information about this feature can be inferred from other features;

2. Purpose of feature selection
It is unknown which feature is valid for a particular learning algorithm. Therefore, relevant features beneficial to the learning algorithm need to be selected from all features. In practice, dimension disaster often occurs. If only part of all features are selected to build the model, the running time of the learning algorithm can be greatly reduced and the interpretability of the model can be increased.

3. Principle of feature selection
The feature subset should be as small as possible, the classification accuracy should not be significantly reduced, the classification distribution should not be affected, and the feature subset should be stable and adaptable.

## Method of feature selection
### 1.Filter method (Filter type)

Feature selection is performed first, and then the learner is trained, so the process of feature selection is independent of the learner. It is equivalent to filtering features first and then training classifiers with feature subsets.

Main idea: "score" each one-dimensional feature, that is, assign weight to each one-dimensional feature, such weight represents the importance of the feature, and then rank according to the weight.

Main methods:

- Chi-squared test
- Information gain
- Correlation coefficient (s)

Advantages: fast running speed, is a very popular feature selection method.

Disadvantages: unable to provide feedback, the standard/specification of feature selection is completed in the feature search algorithm, the learning algorithm can not be transmitted to the feature search algorithm requirements. In addition, a feature may be treated as unimportant for any reason, but may become important in combination with other features.

### 2.Wrapper method
The last classifier is directly used as the evaluation function of feature selection and the optimal feature subset is selected for a specific classifier.

Main idea: consider the selection of subsets as a search optimization problem, generate different combinations, evaluate the combinations, and then compare them with other combinations. In this way, the selection of subsets is regarded as an optimization problem, which can be solved by many optimization algorithms, especially some heuristic optimization algorithms, such as GA, PSO (e.g. optimization algorithm - particle swarm algorithm), DE, ABC (e.g. optimization algorithm - artificial swarm algorithm), etc.

Main method: recursive feature elimination algorithm.

Advantages: feature search is carried out around the learning algorithm, the standard/specification of feature selection is carried out in the requirements of the learning algorithm, can consider the arbitrary learning bias of the learning algorithm, so as to determine the best sub-feature, really pay attention to the learning problem itself. Encapsulation can play a huge role because the learning algorithm has to be run every time a particular subset is tried, so it is possible to pay attention to the learning bias/inductive bias of the learning algorithm.

Disadvantages: running speed is much slower than the filtering algorithm, the actual application of encapsulation method is not popular filtering method.

### 3.Embedded method (Embedded)
If feature selection is embedded in model training, the training may be the same model, but after feature selection is completed, the features completed by feature selection and the superparameters trained by the model can be given to train and optimize again.

Main idea: learn the best features to improve the accuracy of the model under the condition of the established model. That is, in the process of determining the model, the characteristics that are important for the training of the model are selected.

Main methods: feature selection is completed with terms with L1 regularization (which can also be optimized with L2 penalty terms), and random forest mean impurity reduction method/mean accuracy reduction method.

Advantages: The feature search is carried out around the learning algorithm, and any learning bias of the learning algorithm can be considered. The number of times to train the model is smaller than the Wrapper method, which saves time.

Disadvantages: Slow operation.

### Feature selection Method 1: Remove features with low variance.

This method is generally used as a pre-processing work before feature selection, that is, the features with little value change are removed first, and then other feature selection methods are used to select features.

By examining the variance value of the sample under a certain feature, it can be considered that given a threshold value, which features smaller than a certain threshold value can be discarded.

#### 1. Implementation principle
- Discrete variable:
If the eigenvalues of a feature are only 0 and 1, and the value of the feature is 1 in 95% of all the input samples, then the feature is considered insignificant.
If 100% of them are 1's, then this feature is meaningless.
- Continuous variable:
We need to discretize the continuous variables before we can use them.

Moreover, in practice, there are generally not more than 95% of the characteristics of a certain value, so this method is simple but not very useful. It can be used as a pre-processing of feature selection, first removing those features with small value changes, and then selecting appropriate features from the feature selection methods mentioned next for further feature selection.

In [3]:
from sklearn.feature_selection import VarianceThreshold
X = [[0, 0, 1], [0, 1, 0], [1, 0, 0], [0, 1, 1], [0, 1, 0], [0, 1, 1]]
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
sel.fit_transform(X)

array([[0, 1],
       [1, 0],
       [0, 0],
       [1, 1],
       [1, 0],
       [1, 1]])

### Feature selection method two: single variable feature selection

Univariate feature selection method independently measures the relationship between each feature and the response variable. Univariate feature selection can test each feature, measure the relationship between the feature and the response variable, and discard the bad features according to the score. This method is simple, easy to run, easy to understand, usually for understanding data has a good effect (but not necessarily effective for feature optimization, improve generalization ability); There are many improved versions and variations of this method.

#### 1.Pearson Correlation
Pearson's correlation coefficient is the simplest way to understand the relationship between characteristics and response variables. It measures the linear correlation between variables.

That is, the covariance of X_I and X_j is divided by the standard deviation of X_I and the standard deviation of X_J, which can be regarded as a special covariance after eliminating the dimensional influence of the two variables.
Covariance measures the deviation degree of each dimension from its mean. If the value of covariance is positive, it indicates that they are positively correlated; otherwise, they are negatively correlated.
The value range of the results is [-1, 1], -1 represents a complete negative correlation, +1 represents a complete positive correlation, 0 represents no linear correlation, and absolute value represents the strength of correlation.
Standard deviation, also known as mean square deviation, is the arithmetic square root of variance and can reflect the degree of dispersion of a data set.
2) It is mainly used for the screening of continuous features, not for the screening of discrete features.
3) Advantages and disadvantages
Advantages:
Correlation coefficient calculation is fast and easy to calculate, and is often performed immediately after the data is obtained (after cleaning and feature extraction). Pearson correlation coefficient can represent rich relationship, coincidence represents positive and negative relationship, absolute value can represent strength.
Disadvantages:
As a feature ordering mechanism, the correlation coefficient is only sensitive to linear relations. If the relations are nonlinear, even if the two variables have a one-to-one correspondence, the correlation coefficient coefficient may be close to 0.

In [4]:
import numpy as np
from scipy.stats import pearsonr

np.random.seed(2019)
size=1000
x = np.random.normal(0, 1, size)
# 计算两变量间的相关系数
print("Lower noise {}".format(pearsonr(x, x + np.random.normal(0, 1, size))))
print("Higher noise {}".format(pearsonr(x, x + np.random.normal(0, 10, size))))


Lower noise (0.7261901361835682, 1.365052942629593e-164)
Higher noise (0.09357375287756167, 0.003057851441699437)


#### 2. Mutual Information and Maximal Information Coefficient

If the variables are not independent, then we can determine whether they are "close" to being independent of each other by examining the Kullback-Leibler divergence between the joint probability distribution and the product of the marginal probability distribution.

1) Mutual information method

Entropy H (Y) and the conditional entropy H (Y | X) is called the mutual information of the difference between, mutual information and the relationship between the conditional entropy:

I(x,y) = H(x) - H(x|y) = H(y) - H(y|x)

In fact, this is the feature selection rule of ID3 decision tree.

Mutual information method is also used to evaluate the correlation between qualitative independent variables and qualitative dependent variables, but it is not convenient to be directly used in feature selection:

It does not belong to the measurement method, and there is no way to normalize, so the results on different data cannot be compared.
It can only be used for discrete feature selection. Continuous feature selection can only be carried out with mutual information before discretization, and the results of mutual information are very sensitive to the discretization method.

2) Maximum information coefficient method

Since mutual information method is not convenient for feature selection, maximum information coefficient is introduced. The maximal information data first looks for an optimal discrete method, and then converts the mutual information value into a measurement method with the value interval of [0,1].


#### 3. Distance correlation
The distance correlation coefficient is designed to overcome the weakness of Pearson correlation coefficient.

Pearson's correlation coefficient is 0, and we cannot conclude that these two variables are independent (possibly non-linear correlation).

For example, the Pearson correlation between x and x^2 is 0, but the two variables are not independent.

In [7]:
from scipy.spatial.distance import pdist, squareform
import numpy as np

# from numbapro import jit, float32

def distcorr(X, Y):
    """ Compute the distance correlation function

    >>> a = [1,2,3,4,5]
    >>> b = np.array([1,2,9,4,4])
    >>> distcorr(a, b)
    0.762676242417
    """
    X = np.atleast_1d(X)
    Y = np.atleast_1d(Y)
    if np.prod(X.shape) == len(X):
        X = X[:, None]
    if np.prod(Y.shape) == len(Y):
        Y = Y[:, None]
    X = np.atleast_2d(X)
    Y = np.atleast_2d(Y)
    n = X.shape[0]
    if Y.shape[0] != X.shape[0]:
        raise ValueError('Number of samples must match')
    a = squareform(pdist(X))
    b = squareform(pdist(Y))
    A = a - a.mean(axis=0)[None, :] - a.mean(axis=1)[:, None] + a.mean()
    B = b - b.mean(axis=0)[None, :] - b.mean(axis=1)[:, None] + b.mean()

    dcov2_xy = (A * B).sum()/float(n * n)
    dcov2_xx = (A * A).sum()/float(n * n)
    dcov2_yy = (B * B).sum()/float(n * n)
    dcor = np.sqrt(dcov2_xy)/np.sqrt(np.sqrt(dcov2_xx) * np.sqrt(dcov2_yy))
    return dcor

#### 4. Model Based Ranking

The idea is to directly use the machine learning algorithm you want to use to build a prediction model for each individual feature and response variable. If the relationship between features and response variables is nonlinear, there are many alternatives, such as tree-based approaches (decision trees, random forests), or extended linear models. Tree-based approaches are one of the simplest because they can model nonlinear relationships well and do not require much tweaking. But the main thing to avoid is overfitting, so the tree depth should be relatively small, and cross validation should be applied.

#### 5. Chi-square test
The Chi-square test, proposed by Karl Pearson, is a widely used hypothesis testing method for counting data. The chi-square value describes the independence of the two events or the degree to which the actual observed value deviates from the expected value. The larger the chi-square value is, the larger the deviation between the actual observed value of the table name and the expected value is, which also indicates that the independence of the two events is weaker.

1) Principle introduction

The CHI value (CHI square value) is used to measure the difference between the actual value and the theoretical value. In order to avoid the deviation between different observed values and different expectations, it is divided by T, so it is divided by E to eliminate this disadvantage.

The absolute size of the deviation between the actual value and the theoretical value (the difference is magnified by the presence of squares)
The relative size of the difference and the theoretical value

2) Implementation process

The higher the CHI value is, the less likely the two variables are to be independent. In other words, the higher the CHI value is, the higher the correlation degree of the two variables is.
A. For characteristic variables x1,x2... ,xn, and the classification variable y. Just compute CHI(x1,y), CHI(x2,y),... , CHI(xn,y), and sort the features according to the value of CHI from large to small.
B. Select an appropriate threshold. Features larger than the threshold are retained, and features smaller than the threshold are deleted. A subset of features screened out in this way is the feature of input model training.

3) It can only be used for the screening of discrete features in classification problems, but not for the screening of continuous features in classification problems, nor for the feature screening of regression problems.

Realistic approach:

Sklearn. Feature_selection. SelectKBest:
Returns k best features

Sklearn. Feature_selection. SelectPercentile:
Returns the top r% of the best performing characteristics

#### 6. Summary

- The method of removing the features with small value changes is generally used as a pre-processing work before feature selection, that is, removing the features with small value changes first, and then using other feature selection methods to select the features. If the machine has sufficient resources and you want to retain as much information as possible, you can set the threshold high or filter only discrete features with only one value.

- Univariate feature selection can be used to understand data, data structure, characteristics, or to exclude irrelevant features, but it cannot find redundant features.

- The feature method with small value change and the single variable feature selection method belong to the filtering class feature selection method, but the learning algorithm cannot transmit the feature requirement to the feature search algorithm.